In [ ]:
import numpy as np
import cv2
import dlib

detector = dlib.get_frontal_face_detector()
# predictor = dlib.shape_predictor(r'./model/shape_predictor_68_face_landmarks.dat')
predictor = dlib.shape_predictor(r'./model/shape_predictor_5_face_landmarks.dat')
font = 2;    lt = 16            # 利用cv2.putText輸出1-68

img = cv2.imread('./image/face68.JFIF')     # cv2讀取影像
# img = cv2.imread('./image/lenaColor.png')     # cv2讀取影像
img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # 取灰度

rects = detector(img_gray, 0)             # 人臉數rects
for i in range(len(rects)):
    landmarks = np.matrix([[p.x, p.y] for p in predictor(img, rects[i]).parts()])
    # landmarks = np.array([[p.x, p.y] for p in predictor(img,rects[i]).parts()]);   print(landmarks)
    for idx, point in enumerate(landmarks):
        
        pos = (point[0, 0], point[0, 1])   # 68點的座標
#         print(idx,pos)

        cv2.circle(img, pos, 3, color=(0, 255, 0)) # 利用cv2.circle給每個特徵點畫一個圈，共68個
        cv2.putText(img, str(idx+1), pos, font, 0.8, (0, 0, 255), 1, lt)

cv2.imshow('img', img)

cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)

### loading

In [ ]:
# 匯入必要的library
from scipy.io import loadmat
from skimage import io
import dlib, os, glob, re
from imutils import paths
options = dlib.simple_object_detector_training_options()
images = [] # 存放相片圖檔
boxes = []  # 存放Annotations

categories = glob.glob(r'./dlib_ObjectCategories10/**/*.jpg', recursive=True)
annotation = glob.glob(r'./dlib_Annotations10/**/ann*.mat', recursive=True)

for cate, anno in zip(categories, annotation) :
    print(cate, anno)
    xywh = loadmat(anno)['box_coord']
    # 取出annotations資訊繪成矩形物件，放入boxes變數中。
    # (x, y, w, h) = (b.left(), b.top(), b.right(), b.bottom())
    bb = [dlib.rectangle(left=int(x), top=int(y), right=int(w), bottom=int(h)) for (y, h, x, w) in xywh]
    boxes.append(bb)
    images.append(io.imread(cate))          #將圖片放入images變數
print('done', len(images), len(boxes))

### training

In [ ]:
#丟入三個參數開始訓練
options = dlib.simple_object_detector_training_options()
print('[INFO] training detector...')
detector = dlib.train_simple_object_detector(images, boxes, options)
 
# 將訓練結果匯出到檔案
print('[INFO] dumping classifier to file...')
detector.save('./dlib_output/model.svm')           # save

detector = dlib.simple_object_detector('./dlib_output/model.svm')   # load back

# 圖形化顯示 Histogram of Oriented Gradients（簡稱HOG）
win = dlib.image_window()
win.set_image(detector)
# dlib.hit_enter_to_continue()

### detect

In [ ]:
import dlib
import cv2
from imutils import paths
#載入訓練好的detector
# detector = dlib.simple_object_detector(detector)
 
#載入測試圖片逐張進行
# for idx, testingPath in enumerate(paths.list_images('./dlib_ObjectCategories10/accordion')):
for idx, testingPath in enumerate(paths.list_images('./dlib_ObjectCategories10/camera')):
#讀取圖片並執行dector並產生矩形物件以便用於標記辨識出的部份
    image = cv2.imread(testingPath)
    boxes = detector(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    for b in boxes:                    #在圖片上繪出該矩形
        (x, y, w, h) = (b.left(), b.top(), b.right(), b.bottom())
        cv2.rectangle(image, (x, y), (w, h), (0, 255, 0), 2)

    cv2.imshow(str(idx), image)         #顯示圖片
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)